In [50]:
import pandas as pd
import numpy as np

## Reading and Merging Data

In [51]:
# read raw data

asr_12 = pd.read_csv("./data/2012.csv")
asr_16 = pd.read_csv("./data/2016.csv")
asr_20 = pd.read_csv("./data/2020.csv")

In [52]:
# get projected data for only 2020
asr_20 = asr_20[asr_20['year']== 2020]
asr_20['year'] = '2020'

# add year
asr_12['year'] = '2012'
asr_16['year'] = '2016'

In [53]:
# rename columns to fit other dataframes for merging
asr_20.columns = ['year', 'FIPS', 'County', 'Age_num', 'Age',
                  'Total', 'Total Male', 'Total Female', 'Anglo Total',
                  'Anglo Male', 'Anglo Female', 'Black Total', 'Black Male',
                  'Black Female', 'Hispanic Total', 'Hispanic Male', 'Hispanic Female',
                  'Asian Total', 'Asian Male', 'Asian Female', 'Other Total',
                  'Other Male', 'Other Female']

We must combine Asian demographic data with the Other category in asr_20 because the other dataframes do not have data on Asian populations.

In [54]:
asr_20['Other Total'] = asr_20['Other Total'] + asr_20['Asian Total']
asr_20['Other Male'] = asr_20['Other Male'] + asr_20['Asian Male']
asr_20['Other Female'] = asr_20['Other Female'] + asr_20['Asian Female']

In [55]:
asr_20 = asr_20.drop(columns = ['Age_num', 'Asian Total', 'Asian Male', 'Asian Female'])

In [56]:
# combine dataframes
asr_all = pd.concat([asr_12, asr_16, asr_20], axis = 0, ignore_index = True)

## Data Cleaning and Reformatting

In [57]:
# change column name formatting to convention (lowercase and underscores)
cols = [col.lower() for col in asr_all.columns]
cols = [col.replace(" ", "_") for col in cols]
asr_all.columns = cols

# change county info to lower case for easier index merging
asr_all['county'] = asr_all['county'].apply(lambda x: x.lower())
# change de witt county to dewitt county
asr_all['county'] = asr_all['county'].replace(to_replace = "de witt county",
                                             value = "dewitt county")

# remove unnecessary columns - fips
asr_all.drop(columns = ['fips'], inplace = True)

# remove total state population data
mask_all_tx = asr_all['county'] == "state of texas"
asr_all = asr_all[~mask_all_tx]

# keep all age data for later % calculations
asr_all_ages = asr_all[asr_all['age'] == 'All Ages']

# remove all age total population data
asr_all = asr_all[asr_all['age'] != 'All Ages']

In [58]:
asr_all_ages.head()

,county,age,total,total_male,total_female,anglo_total,anglo_male,anglo_female,black_total,black_male,black_female,other_total,other_male,other_female,hispanic_total,hispanic_male,hispanic_female,year
87,anderson county,All Ages,58964,35746,23218,35797,19690,16107,12280,8906,3374,1207,588,619,9680,6562,3118,2012
174,andrews county,All Ages,16039,8007,8032,7512,3667,3845,218,112,106,323,158,165,7986,4070,3916,2012
261,angelina county,All Ages,88850,43635,45215,55351,26960,28391,13188,6239,6949,2021,940,1081,18290,9496,8794,2012
348,aransas county,All Ages,23825,11783,12042,16599,8129,8470,253,144,109,906,445,461,6067,3065,3002,2012
435,archer county,All Ages,8996,4527,4469,8103,4056,4047,33,19,14,160,68,92,700,384,316,2012


In [59]:
def get_age_number(age_str):
    """
    Extract integer age number n from string with partial format "n year(s)" or
    "n yr(s)"
    
    Input
    age_str - string - to extract integer age number from
    
    Output
    int age number from age_str or NaN if not found
    """
    words = age_str.split()
    for i, word in enumerate(words):
        if ("year" in word.lower()) or ("yr" in word.lower()):
            try:
                return int(words[i-1])
            except:
                return int(words[i-1].split("+")[0])
    return np.nan

In [60]:
asr_all['age'] = asr_all['age'].apply(get_age_number)

voting_age = 18
asr_all = asr_all[asr_all['age'] >= voting_age]

In [61]:
def assign_age_group(age, start_age = 18, increment_yr = 10):
    """
    """
    bucket = (age - start_age) // increment_yr
    return f"{start_age + (increment_yr * bucket)} to {start_age + (increment_yr * (bucket+1))-1}"

In [62]:
asr_all['age_group'] = asr_all['age'].apply(assign_age_group)

In [63]:
try:
    asr_all.drop(columns = 'age', inplace = True)
except:
    pass
grouped = asr_all.groupby(['county', 'year', 'age_group']).sum()
grouped = grouped.unstack(level = -2)
grouped = grouped.unstack(level = -1).reset_index()
grouped

county    total                                      \
year                           2012                                       
age_group                  18 to 27 28 to 37 38 to 47 48 to 57 58 to 67   
0          anderson county   7387.0   9358.0   9419.0   8915.0   6285.0   
1           andrews county   2209.0   2050.0   1909.0   2251.0   1473.0   
2          angelina county  11619.0  10862.0  11243.0  11881.0   9576.0   
3           aransas county   2353.0   2019.0   2457.0   3557.0   3908.0   
4            archer county   1022.0    832.0   1156.0   1587.0   1111.0   
..                     ...      ...      ...      ...      ...      ...   
249            wood county   4415.0   3698.0   4444.0   5854.0   6627.0   
250          yoakum county   1040.0   1005.0    941.0   1070.0    746.0   
251           young county   2122.0   1982.0   2185.0   2796.0   2360.0   
252          zapata county   2287.0   1850.0   1614.0   1429.0   1182.0   
253          zavala county   1905.0   1381.0   1313.0   1324.0   1190.0   

                                               ... hispanic_female           \
year                                     2016  ...            2016            
age_group 68 to 77 78 to 87 88 to 97 18 to 27  ...        78 to 87 88 to 97   
0           3832.0   2313.0      NaN   7062.0  ...            36.0      NaN   
1            884.0    621.0      NaN   2450.0  ...            99.0      NaN   
2           6062.0   4098.0      NaN  12110.0  ...           108.0      NaN   
3           3248.0   1753.0      NaN   2458.0  ...           130.0      NaN   
4            776.0    462.0      NaN   1213.0  ...            11.0      NaN   
..             ...      ...      ...      ...  ...             ...      ...   
249         5734.0   3067.0      NaN   4751.0  ...            35.0      NaN   
250          457.0    309.0      NaN   1184.0  ...            52.0      NaN   
251         1601.0   1333.0      NaN   2105.0  ...            27.0      NaN   
252          791.0    471.0      NaN   2208.0  ...           178.0      NaN   
253          704.0    530.0      NaN   1827.0  ...           278.0      NaN   

                                                                          \
year          2020                                                         
age_group 18 to 27 28 to 37 38 to 47 48 to 57 58 to 67 68 to 77 78 to 87   
0            633.0    466.0    460.0    360.0    207.0    153.0     40.0   
1           1054.0   1131.0    809.0    541.0    362.0    213.0     92.0   
2           1654.0   1335.0   1337.0    971.0    666.0    342.0    123.0   
3            621.0    596.0    454.0    410.0    362.0    243.0    121.0   
4             67.0     49.0     42.0     43.0     25.0      7.0     12.0   
..             ...      ...      ...      ...      ...      ...      ...   
249          370.0    304.0    242.0    188.0    181.0     65.0     49.0   
250          465.0    436.0    309.0    309.0    218.0    134.0     71.0   
251          190.0    296.0    219.0    172.0    163.0     56.0     27.0   
252          951.0    940.0    838.0    757.0    604.0    489.0    179.0   
253          842.0    754.0    662.0    581.0    527.0    447.0    191.0   

                    
year                
age_group 88 to 97  
0             20.0  
1             30.0  
2             30.0  
3             31.0  
4              5.0  
..             ...  
249           12.0  
250           20.0  
251           13.0  
252           50.0  
253           64.0  

[254 rows x 361 columns]

In [64]:
grouped.columns = ["_".join(col_tuple) for col_tuple in list(grouped.columns)]

In [65]:
grouped.rename(columns = {'county__' : 'county'}, inplace = True)

In [66]:
def change_to_pct(col_name, ignore_year = False):
    """
    changes given column df[col_name] to percentage of total county population
    rounded to 4 decimal places in format 0.0000
    
    inputs:
    col_name - str - the name of the column to convert
    ignore_year - bool - indicates which dataframe to take data from and compare to
        as well as whether year is relevant
    """
    result = []
    if not ignore_year:
        col_to_convert = grouped[col_name]
        if "2012" in col_name:
            year = "2012"
        elif "2016" in col_name:
            year = "2016"
        else:
            year = "2020"
        total_col = asr_all_ages[asr_all_ages['year'] == year]['total']
    else:
        col_to_convert = asr_all_ages[col_name]
        total_col = asr_all_ages['total']
        
    for x, y in zip(col_to_convert, total_col):
        if np.isnan(x):
            result.append(0)
        else:
            result.append(round(x/y, 4))
    return result

In [67]:
# convert demographic data to percentages out of total pop for county
try:
    grouped.set_index('county', inplace = True)
except:
    pass

for col in grouped.columns:
    grouped[col] = change_to_pct(col)

In [68]:
# convert total m/f data to percentages
convert = ["total_male", "total_female"]
for col in convert:
    asr_all_ages[col] = change_to_pct(col, ignore_year = True)
    
# make new columns in grouped corresponding to total m/f percentage data
county_list = grouped.index
year_list = ["2012", "2016", "2020"]
sex_list = ['male', 'female']

for sex in sex_list:
    for year in year_list:
        data = list(asr_all_ages[asr_all_ages['year'] == year][f"total_{sex}"])
        grouped[f"total_{sex}_{year}"] = data

In [69]:
grouped.shape

(254, 366)

In [70]:
grouped.head()

,total_2012_18 to 27,total_2012_28 to 37,total_2012_38 to 47,total_2012_48 to 57,total_2012_58 to 67,total_2012_68 to 77,total_2012_78 to 87,total_2012_88 to 97,total_2016_18 to 27,total_2016_28 to 37,...,hispanic_female_2020_58 to 67,hispanic_female_2020_68 to 77,hispanic_female_2020_78 to 87,hispanic_female_2020_88 to 97,total_male_2012,total_male_2016,total_male_2020,total_female_2012,total_female_2016,total_female_2020
county,,,,,,,,,,,,,,,,,,,,,
anderson county,0.1253,0.1587,0.1597,0.1512,0.1066,0.0650,0.0392,0,0.1211,0.1489,...,0.0036,0.0026,0.0007,0.0003,0.6062,0.6078,0.6198,0.3938,0.3922,0.3802
andrews county,0.1377,0.1278,0.1190,0.1403,0.0918,0.0551,0.0387,0,0.1374,0.1298,...,0.0163,0.0096,0.0041,0.0013,0.4992,0.5017,0.5158,0.5008,0.4983,0.4842
angelina county,0.1308,0.1223,0.1265,0.1337,0.1078,0.0682,0.0461,0,0.1336,0.1166,...,0.0074,0.0038,0.0014,0.0003,0.4911,0.4950,0.4908,0.5089,0.5050,0.5092
aransas county,0.0988,0.0847,0.1031,0.1493,0.1640,0.1363,0.0736,0,0.0974,0.0814,...,0.0131,0.0088,0.0044,0.0011,0.4946,0.4930,0.4995,0.5054,0.5070,0.5005
archer county,0.1136,0.0925,0.1285,0.1764,0.1235,0.0863,0.0514,0,0.1330,0.0836,...,0.0030,0.0008,0.0014,0.0006,0.5032,0.4956,0.4936,0.4968,0.5044,0.5064


In [71]:
grouped.to_csv("./data/asr_12_16_20.csv", index = True)